In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline



In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



In [ ]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


In [ ]:
# Text preprocessing function
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords, convert to lowercase, lemmatize and remove duplicates
    filtered_tokens = set()
    for token in tokens:
        token = token.lower()
        if token.isalnum() and token not in stop_words:
            lemmatized_token = lemmatizer.lemmatize(token)
            filtered_tokens.add(lemmatized_token)
    return ' '.join(filtered_tokens)



In [ ]:
# Feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Discription'])
y = df['Section']

    

In [ ]:
# Train a classifier
classifier = MultinomialNB()
classifier.fit(X, y)


In [ ]:
# Evaluate the classifier
y_pred = classifier.predict(X)
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# Function to recommend multiple sections based on input Discription in reverse order of probability
def recommend_sections_reverse(Discription, n=2):
    preprocessed_desc = preprocess_text(Discription)
    desc_vectorized = vectorizer.transform([preprocessed_desc])
    
    # Predict probabilities for all classes
    probas = classifier.predict_proba(desc_vectorized)
    
    # Get indices of top n probabilities in reverse order
    top_n_indices = np.argsort(probas, axis=1)[:,-n:][:,::-1]
    
    # Get corresponding section names
    top_n_sections = [classifier.classes_[idx] for idx in top_n_indices[0]]
    
    return top_n_sections



In [ ]:
# Test the recommendation function
Discription = "I seek urgent assistance with a property dispute at [Address]. Despite legal attempts, unlawful occupation persists, disrupting our community. I urge prompt intervention to uphold the rightful owners' rights and restore order. Please contact me for further details. Thank you for your attention."
# Discription='Abetment of any offence, if the act abetted is committed in consequence, and where no express provision is made for its punishment'

recommended_sections_reverse = recommend_sections_reverse(Discription)
print("Recommended Sections (Reverse Order):", recommended_sections_reverse)